In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/wage2.csv')
df['age_2']=df['age']*df['age']
df

,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage,age_2
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2.0,8.0,8.0,6.645091,961
1,808,50,119,41,18,11,16,37,1,0,0,1,1,NaN,14.0,14.0,6.694562,1369
2,825,40,108,46,14,11,9,33,1,0,0,1,1,2.0,14.0,14.0,6.715384,1089
3,650,40,96,32,12,13,7,32,1,0,0,1,4,3.0,12.0,12.0,6.476973,1024
4,562,40,74,27,11,14,5,34,1,0,0,1,10,6.0,6.0,11.0,6.331502,1156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,520,40,79,28,16,6,1,30,1,1,1,0,0,1.0,11.0,NaN,6.253829,900
931,1202,40,102,32,13,10,3,31,1,0,1,1,7,7.0,8.0,6.0,7.091742,961
932,538,45,77,22,12,12,10,28,1,1,1,0,9,NaN,7.0,NaN,6.287858,784
933,873,44,109,25,12,12,12,28,1,0,1,0,1,1.0,NaN,11.0,6.771935,784


In [7]:

wage_eq12 = smf.ols(formula = 'np.log(wage)~age_2+  age + IQ+married + urban + south', data = df).fit()

In [9]:
info_dict = {'No. Observations': lambda x: '{0:d}'.format(int(x.nobs)), 
             'Residual Std. Error': lambda x: "%#8.3f" % x.mse_resid**0.5,
             'F-statistic': lambda x: '{:.3f}'.format(x.fvalue), 
             'Prob (F-statistic)': lambda x: '{:.3f}'.format(x.f_pvalue)
             }
print(summary_col(wage_eq12, float_format='%.3f', stars=True, info_dict = info_dict))


                    np.log(wage)
--------------------------------
Intercept           2.938*      
                    (1.656)     
age_2               -0.002      
                    (0.002)     
age                 0.145       
                    (0.100)     
IQ                  0.008***    
                    (0.001)     
married             0.202***    
                    (0.040)     
urban               0.173***    
                    (0.028)     
south               -0.100***   
                    (0.027)     
R-squared           0.201       
R-squared Adj.      0.196       
No. Observations    935         
Residual Std. Error 0.378       
F-statistic         38.898      
Prob (F-statistic)  0.000       
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [10]:
df = summary_params(wage_eq12, alpha=0.05)
df['significance'] = df.apply(lambda x: 'Значим' if x['P>|t|']<0.05 else 'Незначим', axis=1)
df

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],significance
Intercept,2.937802,1.656234,1.773784,7.642669e-02,-0.312596,6.188201,Незначим
age_2,-0.001857,0.001503,-1.235591,2.169233e-01,-0.004805,0.001092,Незначим
age,0.144754,0.100033,1.447058,1.482184e-01,-0.051564,0.341072,Незначим
IQ,0.008216,0.000841,9.769940,1.584051e-21,0.006566,0.009866,Значим
married,0.201539,0.040240,5.008445,6.567021e-07,0.122567,0.280510,Значим
urban,0.173195,0.027667,6.259939,5.878259e-10,0.118897,0.227492,Значим
south,-0.100273,0.026838,-3.736230,1.982067e-04,-0.152944,-0.047603,Значим
